# Eliminação Gaussiana

Aqui vamos chamar eliminação gaussiana caso o algoritmo transforma a matriz de coeficientes numa matriz triangular superior e eliminação de Gauss-Jordan caso transforme a matriz dos coeficientes numa matriz diagonal.

In [681]:
from typing import Callable
import numpy as np
import matplotlib.pyplot as plt

# Eliminação Gaussiana
Algoritmos para resolver sistemas de equações lineares com eliminação gaussiana.

## Sem Pivotagem

In [682]:
def gausselim(Ao: np.ndarray, bbo: np.ndarray) -> np.ndarray:
    """
        Resolve os M sistemas de N equações definidos por A * xx = bb usando eliminação gaussiana.

        ### Argumentos
        Ao: Matriz N por N
        bbo: Matriz N por M
        
        ### Retorno
        xx: Matriz N por M
    """

    # Dimensões em jogo
    N = Ao.shape[0]

    # Evitar side effects
    A = np.copy(Ao)
    bb = np.copy(bbo)

    # Eliminação Gaussiana
    for i in range(N-1):
        piv = A[i, i]
        for j in range(i+1, N):
            coef = A[j, i] / piv
            A[j] -= A[i] * coef
            bb[j] -= bb[i] * coef

    # Substituição Regressiva
    xx = np.zeros_like(bb)
    xx[-1] = bb[-1] / A[N-1, N-1]
    for i in range(N-1, -1, -1):
        xx[i] = (bb[i] - (A[i, i+1:] @ xx[i+1:])) / A[i, i]
    
    return xx

## Com Pivotagem Parcial

In [683]:
def gausselimpp(Ao: np.ndarray, bbo: np.ndarray) -> np.ndarray:
    """
        Resolve os M sistemas de N equações definidos por A * xx = bb usando eliminação gaussiana com pivotagem parcial.

        ### Argumentos
        Ao: Matriz N por N
        bbo: Matriz N por M
        
        ### Retorno
        xx: Matriz N por M
    """

    # Dimensões em jogo
    N = Ao.shape[0]

    # Evitar side effects
    A = np.copy(Ao)
    bb = np.copy(bbo)


    # Fazer pivotagem usando um mapa de endereços
    mm = np.arange(0, N)
    for i in range(N-1):
        # Posição do maior elemento da coluna (tendo em conta as trocas já feitas)
        coluna = A[mm[i:], i]
        index = list(abs(coluna)).index(max(abs(coluna))) + i

        # Trocar os elementos
        mm[i], mm[index] = mm[index], mm[i]

        # Pivô
        piv = A[mm[i], i]
        
        for j in range(i+1, N):
            coef = A[mm[j], i] / piv
            A[mm[j]] -= A[mm[i]] * coef
            bb[mm[j]] -= bb[mm[i]] * coef

    # Substituição Regressiva
    xx = np.zeros_like(bb)
    xx[-1] = bb[mm[-1]] / A[mm[N-1], N-1]
    for i in range(N-1, -1, -1):
        xx[i] = (bb[mm[i]] - (A[mm[i], i+1:] @ xx[i+1:])) / A[mm[i], i]
    
    return xx


## Com Pivotagem Total

In [684]:
def gausselimpt(Ao: np.ndarray, bbo: np.ndarray) -> np.ndarray:
    """
        Resolve os M sistemas de N equações definidos por A * xx = bb usando eliminação gaussiana com pivotagem total.

        ### Argumentos
        Ao: Matriz N por N
        bbo: Matriz N por M
        
        ### Retorno
        xx: Matriz N por M
    """

    # Dimensões em jogo
    N = Ao.shape[0]

    # Evitar side effects
    A = np.copy(Ao)
    bb = np.copy(bbo)


    # Fazer pivotagem usando dois mapas de endereços (linhas e colunas)
    ll = np.arange(0, N)
    cc = np.arange(0, N)

    for i in range(N-1):
        # Posição do maior elemento da matriz abaixo do pivô (tendo em conta as trocas já feitas)
        maxi = -np.inf
        index = [0, 0]
        for line in range(i, N):
            elements = abs(A[ll[line], cc[i:]])
            colmax = np.argmax(elements)
            if elements[colmax] > maxi:
                maxi = elements[colmax]
                index = [line, colmax + i]

        # Trocar os elementos
        ll[i], ll[index[0]] = ll[index[0]], ll[i]
        cc[i], cc[index[1]] = cc[index[1]], cc[i]

        # Pivô
        piv = A[ll[i], cc[i]]
        
        for j in range(i+1, N):
            coef = A[ll[j], cc[i]] / piv
            A[ll[j]] -= A[ll[i]] * coef
            bb[ll[j]] -= bb[ll[i]] * coef

    # Substituição Regressiva
    # Note-se que quando trocamos duas colunas estamos a trocar a ordem das variáveis das equações!
    xx = np.zeros_like(bb)
    xx[cc[-1]] = bb[ll[-1]] / A[ll[N-1], cc[N-1]]
    for i in range(N-1, -1, -1):
        xx[cc[i]] = (bb[ll[i]] - (A[ll[i], cc[i+1:]] @ xx[cc[i+1:]])) / A[ll[i], cc[i]]
    
    return xx

## Matriz Inversa

In [685]:
def gaussinv(Ao: np.ndarray) -> np.ndarray:
    """
        Inverte a matriz N por N dada usando eliminação gaussiana com pivotagem total.

        ### Argumentos
        Ao: Matriz N por N
        
        ### Retorno
        Ainv: Matriz N por N
    """

    # Resolvemos o sistema AX = I onde I é a matriz identidade, pelo que X será a matriz inversa de A.
    return gausselimpt(Ao, np.eye(Ao.shape[0]))

# Eliminação de Gauss-Jordan
Algoritmos para resolver sistemas de equações lineares com eliminação de Gauss-Jordan.

## Sem Pivotagem

In [686]:
def gaussjordan(Ao: np.ndarray, bbo: np.ndarray) -> np.ndarray:
    """
        Resolve o sistema de D equações definido por A * xx = bb usando eliminação de Gauss-Jordan.
        
        ### Argumentos
        Ao: Matriz N por N
        bbo: Matriz N por M
        
        ### Retorno
        xx: Matriz N por M
    """

    # Dimensões em jogo
    N = Ao.shape[0]
    M = 1 if len(bbo.shape) == 1 else bbo.shape[1]

    bbo.reshape((N, M))

    # Criar a matriz aumentada
    A = np.zeros((N, N + M))
    A[:, :N] = np.copy(Ao)

    if M == 1:
        A[:, N] = np.copy(bbo)
    else:
        A[:, N:] = np.copy(bbo)

    # Eliminação Gaussiana
    for i in range(N):
        piv = A[i, i]

        # Alterar as linhas abaixo do pivô
        for j in range(i+1, N):
            coef = A[j, i] / piv
            A[j] -= A[i] * coef
        
        # Alterar as linhas acima do pivô
        for j in range(0, i):
            coef = A[j, i] / piv
            A[j] -= A[i] * coef
        
        # Normalizar a linha
        A[i] /= piv

    return A[:, N] if M == 1 else A[:, N:]

## Com Pivotagem Parcial

In [687]:
def gaussjordanpp(Ao: np.ndarray, bbo: np.ndarray) -> np.ndarray:
    """
        Resolve o sistema de D equações definido por A * xx = bb usando eliminação de Gauss-Jordan com pivotagem parcial.

        ### Argumentos
        Ao: Matriz N por N
        bbo: Matriz N por M
        
        ### Retorno
        xx: Matriz N por M
    """

    # Dimensões em jogo
    N = Ao.shape[0]
    M = 1 if len(bbo.shape) == 1 else bbo.shape[1]

    bbo.reshape((N, M))

    # Criar a matriz aumentada
    A = np.zeros((N, N + M))
    A[:, :N] = np.copy(Ao)

    if M == 1:
        A[:, N] = np.copy(bbo)
    else:
        A[:, N:] = np.copy(bbo)


    # Fazer pivotagem usando um mapa de endereços
    mm = np.arange(0, N)
    for i in range(N):
        # Posição do maior elemento da coluna (tendo em conta as trocas já feitas)
        coluna = A[mm[i:], i]
        index = list(abs(coluna)).index(max(abs(coluna))) + i

        # Trocar os elementos
        mm[i], mm[index] = mm[index], mm[i]

        # Pivô
        piv = A[mm[i], i]

        # Alterar as linhas abaixo do pivô
        for j in range(i+1, N):
            coef = A[mm[j], i] / piv
            A[mm[j]] -= A[mm[i]] * coef
        
        # Alterar as linhas acima do pivô
        for j in range(0, i):
            coef = A[mm[j], i] / piv
            A[mm[j]] -= A[mm[i]] * coef
        
        # Normalizar a linha
        A[mm[i]] /= piv

    return A[mm[:], N] if M == 1 else A[mm[:], N:]

# Testar

Vamos fazer uma bateria de testes a este métodos!

## Resolver Equações Lineares

In [688]:
from testes import testelinear, testeinv

In [689]:
print("Eliminação Gaussiana\n")
testelinear(gausselim, 10)

Eliminação Gaussiana

Para um sistema aleatório este método difere do resultado do numpy no máximo 1.399e-14.

Para cinco sistemas aleatórios este método difere do resultado do numpy no máximo 1.039e-13.



In [690]:
print("Eliminação Gaussiana com Pivotagem Parcial\n")
testelinear(gausselimpp, 10)

Eliminação Gaussiana com Pivotagem Parcial

Para um sistema aleatório este método difere do resultado do numpy no máximo 9.770e-15.

Para cinco sistemas aleatórios este método difere do resultado do numpy no máximo 3.109e-15.



In [691]:
print("Eliminação Gaussiana com Pivotagem Total\n")
testelinear(gausselimpt, 10)

Eliminação Gaussiana com Pivotagem Total

Para um sistema aleatório este método difere do resultado do numpy no máximo 1.221e-15.

Para cinco sistemas aleatórios este método difere do resultado do numpy no máximo 1.776e-15.



In [692]:
print("Eliminação de Gauss-Jordan\n")
testelinear(gaussjordan, 10)

Eliminação de Gauss-Jordan

Para um sistema aleatório este método difere do resultado do numpy no máximo 1.243e-14.

Para cinco sistemas aleatórios este método difere do resultado do numpy no máximo 1.205e-14.



In [693]:
print("Eliminação de Gauss-Jordan com pivotagem parcial\n")
testelinear(gaussjordanpp, 10)

Eliminação de Gauss-Jordan com pivotagem parcial

Para um sistema aleatório este método difere do resultado do numpy no máximo 4.663e-15.

Para cinco sistemas aleatórios este método difere do resultado do numpy no máximo 1.110e-15.



## Inverter Matrizes

In [694]:
print("Eliminação Gaussiana com Pivotagem Parcial\n")
testeinv(gaussinv, 10)

Eliminação Gaussiana com Pivotagem Parcial

Para uma matriz aleatória este método difere do resultado do numpy no máximo 2.220e-15.

